In [ ]:
import json
from copy import deepcopy
from dataclasses import dataclass, field
from enum import Enum
from typing import Any, Callable, Dict, List, Optional, Sequence, Type, TypeVar, Union, get_args

# from pydantic_core import ValidationError, core_schema
# from pydantic import BaseModel, GetCoreSchemaHandler
import logfire
import nest_asyncio
from dotenv import load_dotenv
from IPython.display import Markdown
from pydantic import BaseModel, Field
from pydantic_ai import Agent
from pydantic_ai.agent import EndStrategy
from pydantic_ai.models import KnownModelName
from pydantic_ai.result import ResultData
from pydantic_ai.settings import ModelSettings
from pydantic_ai.tools import AgentDeps, Tool, ToolFuncContext, ToolFuncEither

load_dotenv()

nest_asyncio.apply()
logfire.configure(send_to_logfire="if-token-present")

In [2]:
def get_datetime(fromat: str = "%Y-%m-%d %H:%M:%S") -> str:
    """Get the current datetime as a string in the specified python format."""
    from datetime import datetime

    return datetime.now().strftime(fromat)

In [3]:
@dataclass
class ToolDef:
    name: str = field(default="None", repr=True)
    function: Optional[Tool] = field(default=None, repr=False, init=False)

    def __post_init__(self):
        if self.function:
            if isinstance(self.function, Tool):
                self.name = self.function.name
            elif isinstance(self.function, Callable):
                self.name = function.__name__
        elif self.name:
            self.function = globals()[self.name]

    def __call__(self):
        return self.function


# TypeVar("ToolList", bound=List[str])
ToolList = TypeVar("ToolList", bound=List[ToolDef])


@dataclass
class ToolSet:
    # tools: ToolList = field(init=True, default_factory=list, repr=True)
    tools: List[ToolDef] = field(init=True, default_factory=list, repr=True)
    # tools: list[ToolList] = field(init=True, default_factory=list, repr=False)

    def __post_init__(self):
        tools = deepcopy(self.tools)
        self.tools = []
        for tool in tools:
            self.add(tool)

    def add(self, tool: Tool):
        if isinstance(tool, Callable):
            tool = ToolDef(
                name=tool.__name__,
            )
        elif isinstance(tool, str):
            tool = ToolDef(
                name=tool,
            )
        else:
            raise ValueError("Tool must be a callable or a string")
        self.tools.append(tool)

    def remove(self, name: str) -> Tool:
        tool = self.get(name)
        if tool:
            self.tools.remove(tool)
        return tool

    def get(self, name: str, default=None):
        return next((tool.function for tool in self.tools if tool.name == name), default)

    def __iter__(self):
        return iter(self.tools)

    def __getitem__(self, item):
        return self.get(item)

    def __len__(self):
        return len(self.tools)

    def __contains__(self, value):
        if isinstance(value, str):
            return self.get(value) is not None
        elif isinstance(value, Callable):
            return value in self.tools
        else:
            raise ValueError("value must be a type of string or callable")

    def __str__(self):
        return str(self.tools)

    def all(self):
        return [tool.name for tool in self.tools]

In [ ]:
def test(_: int = 0):
    """Test tool."""
    return "passed"


tset = ToolSet()
print(f"{tset=}", end="\n\n")

try:
    tset = ToolSet([Tool(test)])
except ValueError as e:
    print(f"{e=}", end="\n\n")

tset.add(test)
print(f"{tset=}", end="\n\n")

tset = ToolSet(["test"])
print(f"{tset=}", end="\n\n")

tset = ToolSet([test])
print(f"{tset=}", end="\n\n")

t = tset.get("test")
print(f"{t()=}", end="\n\n")

print(f"{tset["test"]()=}", end="\n\n")

In [5]:
# @dataclass
# class AgentParams:
#     result_type: Type[ResultData] = str
#     deps_type: type[AgentDeps] = type(None)
#     model_settings: Union[dict, None] = None
#     retries: int = 1
#     result_tool_name: str = "final_result"
#     result_tool_description: Union[str, None] = None
#     result_retries: Union[int, None] = None
#     defer_model_check: bool = False
#     end_strategy: EndStrategy = "early"


@dataclass
class AgentParams:
    params: dict[str, Any]
    # result_type: type[ResultData] = str,
    # deps_type: type[AgentDeps] = NoneType,
    # model_settings: ModelSettings | None = None,
    # retries: int = 1,
    # result_tool_name: str = 'final_result',
    # result_tool_description: str | None = None,
    # result_retries: int | None = None,
    # tools: Sequence[Tool[AgentDeps] | ToolFuncEither[AgentDeps, ...]] = (),
    # defer_model_check: bool = False,
    # end_strategy: EndStrategy = 'early',


@dataclass
class AgentDef:
    model: KnownModelName
    name: str
    system_prompt: str
    params: Optional[AgentParams] = field(default_factory=dict, repr=False)

In [15]:
@dataclass
class TaskDef:
    name: str
    description: str
    priority: int
    agent: AgentDef = field(default=None, init=True, repr=False)
    toolset: Optional[ToolSet] = field(default_factory=ToolSet, init=True, repr=False)

    def __lt__(self, other):
        return self.priority < other.priority

In [16]:
def avilable_agents() -> list[AgentDef]:
    a1 = AgentDef(
        model="openai:gpt-4o-mini",
        name="planner",
        system_prompt="You are a planner. your goal is to make a step by step plan for other agents. Do not answer the user questions. Just make a plan how to do this.",
        params={
            "result_type": "TaskDef",
            "retries": "3",
        },
    )
    a2 = AgentDef(
        model="openai:gpt-4o-mini",
        name="date agent",
        system_prompt="You are date agent.",
    )
    return [a1, a2]

In [ ]:
@dataclass
class Task:
    agent: Agent = field(init=False, repr=False)
    task: TaskDef
    agent_def: AgentDef
    toolset: Optional[ToolSet] = field(default=None, init=True, repr=True)

    def __post_init__(self):
        self.agent = Agent(
            model=self.agent_def.model,
            name=self.agent_def.name,
            system_prompt=self.agent_def.system_prompt,
            tools=self.toolset,
            **self.agent_def.params if self.agent_def.params else {},
        )

    async def run(self):
        return await self.agent.run(self.task.description)

    def run_sync(self):
        return self.agent.run_sync(self.task.description)


print(f"{get_datetime()=}", end="\n\n")

t = Task(
    task=TaskDef(name="get_datetime", description="make a plan on how to get the current datetime", priority=1),
    agent_def=AgentDef(
        model="openai:gpt-4o-mini",
        name="planner",
        system_prompt="You are a planner. your goal is to make a step by step plan for other agents. Get the list of available agents by calling 'agents_available' or create a new agent . Do not answer the user questions. Just make a plan how to do this.",
        params={
            "result_type": AgentDef,
            "retries": 3,
        },
    ),
    toolset=[get_datetime],
)


@t.agent.tool_plain
def tools_available():
    return [Tool(name="get_datetime", description="make a plan on how to get the current datetime", priority=1)]


@t.agent.tool_plain
def agents_available():
    "get list of available agents"
    return [
        AgentDef(
            model="openai:gpt-4o-mini",
            name="date agent",
            system_prompt="You are a date agent.",
        ),
        AgentDef(
            model="openai:gpt-4o-mini",
            name="planner",
            system_prompt="You are a planner. your goal is to make a step by step plan for other agents. Do not answer the user questions. Just make a plan how to do this.",
        ),
    ]


print(f"{t=}", end="\n\n")
# print(f"{task=}", end="\n\n")
print(f"{t.run_sync().data=}", end="\n\n")

In [ ]:
@dataclass
class TaskList:
    tasks: list[TaskDef]

    def __iter__(self):
        return iter(self.tasks)

    def append(self, task: TaskDef):
        self.tasks.append(task)

    # @classmethod
    # def __get_pydantic_core_schema__(cls, source: Any, handler: GetCoreSchemaHandler) -> core_schema.CoreSchema:
    #     instance_schema = core_schema.is_instance_schema(cls)

    #     args = get_args(source)
    #     if args:
    #         # replace the type and rely on Pydantic to generate the right schema
    #         # for `Sequence`
    #         sequence_t_schema = handler.generate_schema(List[args[0]])
    #     else:
    #         sequence_t_schema = handler.generate_schema(List)

    #     non_instance_schema = core_schema.no_info_after_validator_function(TaskList, sequence_t_schema)
    #     return core_schema.union_schema([instance_schema, non_instance_schema])


tl = TaskList(
    [
        TaskDef(
            name="test3",
            description="test 3",
            priority=3,
            agent=AgentDef(name="test3", model="openai:gpt-4o-mini", system_prompt="You are agent 3"),
        ),
        TaskDef(name="test1", description="test 1", priority=1),
        TaskDef(name="test2", description="test 2", priority=2),
    ]
)

print(f"{tl=}")
print(f"{sorted(tl)=}")

In [ ]:
toolset = ToolSet([get_datetime])
# toolset = [get_datetime]
print(f"{toolset=}", end="\n\n")

agent = AgentDef(
    model="openai:gpt-4o-mini",
    name="planner",
    system_prompt="You are a planner. your goal is to make a step by step plan for other agents. Do not answer the user questions. Just make a plan how to do this.",
    params={"result_type": TaskList, "retries": 3},
)
print(f"{agent=}", end="\n\n")

task_def = TaskDef(name="get_datetime", description="make a plan on how to get the current datetime", priority=1)
print(f"{task_def=}", end="\n\n")


task = Task(task=task_def, agent_def=agent, toolset=toolset)
# print(f"{task=}", end="\n\n")


# @task.agent.tool_plain
# def tools_available():
#     """get list of available tools"""
#     return Toolset([get_datetime])


# @task.agent.tool_plain
# def agents_available():
#     """get list of available agents"""
#     return [
#         AgentDef(
#             model="openai:gpt-4o-mini",
#             name="date agent",
#             system_prompt="You are a date agent.",
#         ),
#         AgentDef(
#             model="openai:gpt-4o-mini",
#             name="planner",
#             system_prompt="""You are a planner. your goal is to make a step by step plan for other agents.
#             Get the list of available agents by calling 'agents_available' or create a new agent.
#             You must assign an agent and toolset to every task.
#             Do not answer the user questions. Just make a plan how to do this.""",
#         ),
#     ]


result = task.run_sync()

print(f"{result=}", end="\n\n")
print(f"{result.data=}", end="\n\n")

t = result.data[0]

test_task = Task(task=t, agent_def=t.agent, toolset=t.toolset)
test_result = test_task.run_sync()
Markdown(test_result.data)